<div align="right"><i>Matías Torres Esteban<br>Diciembre, 2025</i></div>

# Interpretación de Textos con LLMs

Los grandes modelos de lenguaje (LLMs) son conocidos por su capacidad de aprender patrones complejos de los textos y de resolver en consecuencia varias tareas de procesamiento del lenguaje natural, como lo son la traducción automática, la generación de resúmenes y la resolución de preguntas de dominios especializados. Es por esto que han surgido nuevas lineas de investigación que buscan utilizar a los grandes modelos de lenguaje (LLMs) como *interpretadores semánticos de textos*. Su objetivo es representar el significado de un texto escrito en lenguaje natural en uno o más lenguajes lógicos —por ejemplo, la lógica proposicional o la lógica de primer orden— para explicitar la información contenida allí y brindarle mayor estructura. De esta manera, podemos obtener representaciones avanzadas del significado de un texto que sean más faciles de almacenar en una computadora y puedan ser manipuladas algorítmicamente.

Un lenguaje lógico que podemos utilizar para representar la información contenida en un texto es el lenguaje de los *Grafos de Conocimiento (KGs)*, los cuales permiten codificar hechos y proposiciones del mundo mediante ternas $(f,r,o)$ donde $f$ denota un concepto fuente, $r$ una relación semántica y $o$ un concepto objetivo. Así, la información contenida en el siguiente texto:

* *"El Covid-19 es una enfermedad infecciosa causada por el SARS-CoV-2. Produce síntomas que incluyen fiebre, tos y fatiga"*,

puede representarse como la siguiente colección de ternas de conocimiento:

* *(Covid-19, es una, Enfermedad Infecciosa)*
* *(Covid-19, causada por, SARS-CoV-2)*.
* *(Covid-19, tiene síntoma, Fiebre)*.
* *(Covid-19, tiene síntoma, Tos)*.
* *(Covid-19, tiene síntoma, Fatiga)*.

Esta colección de ternas puede representase como un grafo dirigido y etiquetado, como se muestra en la siguiente figura:

![GrafoConocimiento](https://raw.githubusercontent.com/matizzat/InforSanLuis-2025-LLMs/4bd9810f4bbfea827eedce1913332bcfb43fd40a/grafo_conocimiento_covid19.svg)

Vemos que este tipo de representación es más rica que el texto puro porque explicita los conceptos y relaciones más relevantes. De esta manera, la interpretación y el análisis de la información extraída se vuelven más accesibles tanto para un usuario como para una computadora, y además se facilita su almacenamiento en bases de datos. Finalmente, si pudiéramos procesar automáticamente un gran conjunto de documentos de un dominio especializado —como la biología o la medicina— y transformarlos en grafos de conocimiento, podríamos aplicar todo el aparato matemático de la Teoría de Grafos para analizar estos sistemas conceptuales y revelar información del dominio que está escondida en los textos.

En esta notebook utilizaremo al modelo Gemini junto a código Python para crear automáticamente grafos de conocimiento a partir de textos. Este ejercicio nos enseñará a coordinar diferentes invocaciones al modelo y a escribir correctamente nuestros prompts para sacarle el máximo provecho posible.

## Proceso de Interpretación

Vamos a implementar un procedimiento de interpretación textual inspirado en la metodología propuesta por Joseph Novak para la construcción de mapas conceptuales [1]. Este procedimiento permite generar un KG a partir de un texto expositivo mediante una secuencia estructurada de 4 pasos:

1. Solicitamos al modelo que analice el texto ``<texto>`` y genere una pregunta de enfoque ``<pregunta>``. Las ternas de conocimiento generadas en los
próximos pasos deberı́an ayudar a responder esta pregunta.

2. Solicitamos al modelo que analice ``<texto>`` y ``<pregunta>`` y que luego genere una lista de conceptos ``<conceptos>``. Los conceptos extraı́dos deben
estar explicitamente mencionados en el texto y tienen que ayudar a responder la pregunta de enfoque.

3. Solictamos al modelo que analice ``<texto>``, ``<pregunta>`` y ``<conceptos>`` y que genere una lista de relaciones semánticas ``<relaciones>``.

4. Solicitamos al modelo que analice ``<texto>``, ``<pregunta>``, ``<conceptos>`` y ``<relaciones>`` en conjunto y que luego genere una lista de ternas de conocimiento ``<ternas>``.

En cada paso realizamos un procesamiento de las etiquetas de conceptos y relaciones para convertir todos sus caracteres a minúsculas y eliminar espacios en blanco innecesarios.

## Código

**Advertencia:** Si la API de Gemini no está disponible en este momento pueden simular el proceso en ChatGPT o el chat de Gemini y reintentar ejecutar el código más tarde.

Primero extraemos los recursos desde Github (textos y prompts):

In [1]:
!git clone https://github.com/matizzat/InforSanLuis-2025-LLMs
%cd InforSanLuis-2025-LLMs

Cloning into 'InforSanLuis-2025-LLMs'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 35.54 KiB | 2.54 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/InforSanLuis-2025-LLMs


Instalamos la librería Pyvis para visualizar grafos de conocimiento:

In [2]:
!pip install pyvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.4 MB/s eta 0:00:00


Importamos las librerías necesarias

In [3]:
from pyvis.network import Network
from google.colab import userdata
from pyvis import network as net
from google.genai import types
from google import genai
from typing import List
import networkx as nx
import pprint
import json
import re

Instanciamos un cliente para invocar al modelo Gemini. Para ejecutar esta celda deben obtener una clave del siguiente [enlace](https://ai.google.dev/gemini-api/docs/api-key). También tienen que configurar Google Colab para utilizar esta clave (Ver  [Tutorial](https://www.youtube.com/watch?v=snrvP_TZjvw))

In [4]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai_client = genai.Client(api_key=GOOGLE_API_KEY)

Definimos una función auxiliar para abrir el contenido de un archivo y cargamos las instrucciones con las que invocaremos al modelo. En cada invocación nosotros le proveemos al modelo una **instrucción de sistema** y una **instrucción de usuario**:

* En la instrucción de sistema nosotros le especificamos al modelo cual es el rol que debe cumplir, le explicamos brevemente la tarea a resolver y le brindamos algunos ejemplos de cómo debe resolver la tarea y formatear sus respuestas.
* En la instrucción de usuario

In [61]:
def abrir_contenido_archivo(nombre_archivo: str):
    with open(nombre_archivo, 'r') as f:
        return f.read()

crear_pregunta_sistema   = abrir_contenido_archivo('./instrucciones/crear_pregunta_sistema.txt')
crear_conceptos_sistema  = abrir_contenido_archivo('./instrucciones/crear_conceptos_sistema.txt')
crear_relaciones_sistema = abrir_contenido_archivo('./instrucciones/crear_relaciones_sistema.txt')
crear_ternas_sistema     = abrir_contenido_archivo('./instrucciones/crear_ternas_sistema.txt')

crear_pregunta_usuario   = abrir_contenido_archivo('./instrucciones/crear_pregunta_usuario.txt')
crear_conceptos_usuario  = abrir_contenido_archivo('./instrucciones/crear_conceptos_usuario.txt')
crear_relaciones_usuario = abrir_contenido_archivo('./instrucciones/crear_relaciones_usuario.txt')
crear_ternas_usuario     = abrir_contenido_archivo('./instrucciones/crear_ternas_usuario.txt')

Ejecuten la siguiente celda probando distintos valores para visualizar las instrucciones con las que vamos a invocar al modelo. Observar que las instrucciones del usuario funcionan como plantillas: las completamos dinámicamente a medida que el LLM avance en cada paso del proceso. Los espacios a modificar dinámicamente están encerrados entre llaves ``{`` y ``}``.

In [40]:
print(crear_conceptos_sistema)

Eres un creador de mapas conceptuales que analiza textos científicos y extrae de ellos los conceptos y relaciones más importantes. Un concepto es un patrón o regularidad en objetos, eventos o registros de objetos y eventos, designado con una etiqueta. Los conceptos se relacionan entre sí mediante frases significativas que forman proposiciones. Te daré un texto de conocimiento delimitado por @ con la siguiente estructura:
    1. Pregunta Focal: Una pregunta que debes responder enumerando los conceptos relevantes.
    2. Conocimiento: Un texto que responde la pregunta focal utilizando conceptos relevantes. Los conceptos que extraigas deben estar explícitamente mencionados o derivados de este texto.
Tu tarea es escribir una lista de conceptos que luego utilizaremos para responder la pregunta focal. Los conceptos deben estar explícitamente mencionados o derivados del Texto de Conocimiento. Incluye los conceptos mencionados en la Pregunta Focal.

Ejemplos:

Texto de Conocimiento:
@
Pregunta

Definimos funciones auxiliares que necesitaremos en nuestro proceso:
* Abrir el cuerpo de documentos.
* ``invocar_llm`` recibe como parámetros una instrucción de sistema y una instrucción de usuario y con ellos invoca al modelo Gemini 2.5 Flash. La invocación esta fijada con un valor de temperatura bajo para que las respuestas del modelo sean determinísticas y no varíen en diferentes invocaciones.  
* Almacenar un conjunto de grafos de conocimiento en un archivo JSON.
* Dibujar un conjunto de grafos de conocimiento en un archivo HTML.

In [27]:
def invocar_llm(sistema: str, usuario: str):
    """
    Invoca al modelo de lenguaje Gemini 2.5 Flash.

    Parámetros:
        sistema (str): Instrucciones del sistema para el modelo.
        usuario  (str): Consulta o instrucciones del usuario.

    Retorna:
        str: Texto generado por el modelo.
    """
    global genai_client

    respuesta = genai_client.models.generate_content(
        config = types.GenerateContentConfig(
            system_instruction = sistema,
            temperature = 0.1
        ),
        model = "gemini-2.5-flash",
        contents = usuario
    )

    return respuesta.text


def abrir_lote_de_documentos(ruta_entrada: str) -> list[dict]:
    """
    Abre un lote de documentos desde un archivo JSON.

    Parámetros:
        ruta_entrada (str): Ruta del archivo que contiene los documentos.

    Retorna:
        list[dict]: Lista de documentos.
    """
    with open(ruta_entrada, 'r') as f:
        textos = json.load(f)
    return textos


def guardar_mapa_conceptual(ruta_salida: str, mapas: dict):
    """
    Guarda una lista de mapas conceptuales en un archivo JSON.

    Parámetros:
        ruta_salida (str): Ruta del archivo de salida.
        mapas (dict): Mapas conceptuales a guardar.
    """
    with open(ruta_salida, 'w') as f:
        json.dump(mapas, f, indent=4)


def dibujar_mapa_conceptual(mapas: dict, ruta_salida: str):
    """
    Crea una visualización pyvis de una lista de mapas conceptuales
    y la guarda como archivo HTML.

    Parámetros:
        mapas (dict): Lista de mapas conceptuales.
        ruta_salida (str): Archivo HTML donde se guardará la visualización.
    """
    G = nx.MultiDiGraph()

    for mapa in mapas:
        pregunta_focal = mapa['fq']
        ternas = mapa['ts']

        for terna in ternas:
            s = terna['s']
            r = terna['r']
            t = terna['t']

            if s not in G:
                G.add_node(s, label=s, color='black')
            if t not in G:
                G.add_node(t, label=t, color='black')

            G.add_edge(
                s,
                t,
                label=r,
                title=pregunta_focal,
                color='blue'
            )

    vis = net.Network(directed=True)
    vis.from_nx(G)
    vis.save_graph(ruta_salida)


Abrimos el lote de documentos y mostramos el texto del primer documento que está almacenado allí. Si revisan el archivo `documentos.json` verán que cada documento consiste de un título y un texto.

In [33]:
documentos = abrir_lote_de_documentos('./documentos.json')
texto = documentos[0]['texto']
print(texto)

El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.


Definimos una función auxiliar para normalizar la etiqueta de un concepto o una relación semántica. Esta elimina el carácter especial `@`, quita los espacios en blanco innecesarios y convierte todas las letras a minúsculas

In [30]:
def normalizar_etiqueta(etiqueta: str):
    etiqueta = etiqueta.replace('@','')
    etiqueta = etiqueta.strip()
    etiqueta = " ".join(etiqueta.split())
    etiqueta = etiqueta.lower()
    return etiqueta

Formateamos el prompt de usuario para instruirle al modelo que genere una pregunta de enfoque.

In [36]:
instruccion_crear_pregunta_formateada = crear_pregunta_usuario.format(texto = texto)
print(instruccion_crear_pregunta_formateada )

Texto de Conocimiento:
@
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
@
Pregunta Focal:


Invocamos al modelo de lenguaje y le solicitamos que cree una pregunta focal para el texto dado.

In [41]:
pregunta_focal = invocar_llm(
    usuario = instruccion_crear_pregunta_formateada,
    sistema = crear_pregunta_sistema)

pregunta_focal = normalizar_etiqueta(pregunta_focal)
print(pregunta_focal)

¿qué es el agua y cuál es su composición molecular?


Formateamos el prompt de usuario para instruirle al modelo que genere una lista de conceptos.

In [43]:
instruccion_crear_conceptos_formateada = crear_conceptos_usuario.format(fq = pregunta_focal, kt = texto)
print(instruccion_crear_conceptos_formateada)

Texto de Conocimiento:
@
Pregunta Focal:
¿qué es el agua y cuál es su composición molecular?
Conocimiento:
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
@
Lista de Conceptos:



Invocamos al modelo de lenguaje para que genere una lista de conceptos a partir del texto y la pregunta de enfoque:

In [48]:
respuesta_llm = invocar_llm(
    usuario = instruccion_crear_conceptos_formateada,
    sistema = crear_conceptos_sistema)

print(respuesta_llm)

Agua
Composición molecular
Sustancia
Molécula
Átomos de hidrógeno
Átomos de oxígeno
H2O
Enlace covalente


Convertimos el texto extraido en una lista de Python:

In [47]:
# Expresión regular para extraer una lista de conceptos:
conceptos_exp_reg = r'[\w\d].*?\n|[\w\d].*$'

lista_conceptos = re.findall(conceptos_exp_reg, respuesta_llm)
lista_conceptos = [normalizar_etiqueta(concepto) for concepto in lista_conceptos]

pprint.pprint(lista_conceptos)

['agua',
 'composición molecular',
 'sustancia',
 'molécula',
 'átomos de hidrógeno',
 'átomo de oxígeno',
 'h2o',
 'enlace covalente']


Formateamos el prompt de usuario para instruirle al modelo que genere una lista de relaciones semánticas:

In [58]:
instruccion_crear_relaciones_formateada = crear_relaciones_usuario.format(
    fq = pregunta_focal,
    cs = "\n".join(lista_conceptos),
    kt = texto)
print(instruccion_crear_relaciones_formateada)

Texto de Conocimiento:
@
Pregunta Focal:
¿qué es el agua y cuál es su composición molecular?
Lista de Conceptos:
agua
composición molecular
sustancia
molécula
átomos de hidrógeno
átomo de oxígeno
h2o
enlace covalente
Conocimiento:
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
@
Relaciones Semánticas:


Invocamos al modelo de lenguaje para que genere una lista de relaciones a partir del texto, la pregunta de enfoque y la lista de conceptos:

In [55]:
respuesta_llm = invocar_llm(
    sistema = crear_relaciones_sistema,
    usuario = instruccion_crear_relaciones_formateada)

print(respuesta_llm)

Es una
Está compuesta por
Unidos por


Convertimos el texto extraido en una lista de Python:

In [56]:
# Expresión regular para extraer una lista de relaciones semánticas:
relaciones_exp_reg = r'[\w\d].*?\n|[\w\d].*$'

lista_relaciones = re.findall(relaciones_exp_reg, respuesta_llm)
lista_relaciones = [normalizar_etiqueta(relacion) for relacion in lista_relaciones]

pprint.pprint(lista_relaciones)

['es una', 'está compuesta por', 'unidos por']


Formateamos el prompt de usuario para instruirle al modelo que genere una lista de ternas de conocimiento:

In [62]:
instruccion_crear_ternas_formateada = crear_ternas_usuario.format(
    fq = pregunta_focal,
    cs = "\n".join(lista_conceptos),
    rs = "\n".join(lista_relaciones),
    kt = texto)

print(instruccion_crear_ternas_formateada)

Texto de Conocimiento:
@
Pregunta Focal:
¿qué es el agua y cuál es su composición molecular?
Lista de Conceptos:
agua
composición molecular
sustancia
molécula
átomos de hidrógeno
átomo de oxígeno
h2o
enlace covalente
Lista de Relaciones:
es una
está compuesta por
unidos por
Conocimiento:
El agua es una sustancia cuya molécula está compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O) unidos por un enlace covalente.
@
Tripletas de Conocimiento:


Invocamos al modelo de lenguaje para que genere una lista de ternas de conocimiento  a partir del texto, la pregunta de enfoque, la lista de conceptos y la lista de relaciones semánticas.

In [64]:
respuesta_llm = invocar_llm(
    sistema = crear_ternas_sistema,
    usuario = instruccion_crear_ternas_formateada)

print(respuesta_llm)

@! agua @ es una @ sustancia !@
@! agua @ está compuesta por @ átomos de hidrógeno !@
@! agua @ está compuesta por @ átomo de oxígeno !@
@! h2o @ es una @ composición molecular !@
@! átomos de hidrógeno @ unidos por @ enlace covalente !@
@! átomo de oxígeno @ unidos por @ enlace covalente !@


Extraemos las ternas de conocimiento del modelo.

In [70]:
ternas_exp_reg = r'@! (.+?) @ (.+?) @ (.+?) !@'

L = re.findall(ternas_exp_reg, respuesta_llm)

lista_ternas = []

for s, r, t in L:
    lista_ternas.append({
        's': normalizar_etiqueta(s),
        'r': normalizar_etiqueta(r),
        't': normalizar_etiqueta(t)
    })

pprint.pprint(lista_ternas)

[{'r': 'es una', 's': 'agua', 't': 'sustancia'},
 {'r': 'está compuesta por', 's': 'agua', 't': 'átomos de hidrógeno'},
 {'r': 'está compuesta por', 's': 'agua', 't': 'átomo de oxígeno'},
 {'r': 'es una', 's': 'h2o', 't': 'composición molecular'},
 {'r': 'unidos por', 's': 'átomos de hidrógeno', 't': 'enlace covalente'},
 {'r': 'unidos por', 's': 'átomo de oxígeno', 't': 'enlace covalente'}]


Unimos todas las componentes obtenidas en un único diccionario Python que representa el grafo de conocimiento (o mapa conceptual):

In [74]:
mapa_conceptual = {'fq': pregunta_focal, 'ts': lista_ternas, 'cs': lista_conceptos, 'rs': lista_relaciones}
pprint.pprint(mapa_conceptual)

{'cs': ['agua',
        'composición molecular',
        'sustancia',
        'molécula',
        'átomos de hidrógeno',
        'átomo de oxígeno',
        'h2o',
        'enlace covalente'],
 'fq': '¿qué es el agua y cuál es su composición molecular?',
 'rs': ['es una', 'está compuesta por', 'unidos por'],
 'ts': [{'r': 'es una', 's': 'agua', 't': 'sustancia'},
        {'r': 'está compuesta por', 's': 'agua', 't': 'átomos de hidrógeno'},
        {'r': 'está compuesta por', 's': 'agua', 't': 'átomo de oxígeno'},
        {'r': 'es una', 's': 'h2o', 't': 'composición molecular'},
        {'r': 'unidos por',
         's': 'átomos de hidrógeno',
         't': 'enlace covalente'},
        {'r': 'unidos por', 's': 'átomo de oxígeno', 't': 'enlace covalente'}]}


Almacenamos en un archivo JSON el grafo de conocimiento obtenido y lo visalizamos en un archivo HTML 😀

In [76]:
guardar_mapa_conceptual('./mapa_conceptual.json', [mapa_conceptual])
dibujar_mapa_conceptual([mapa_conceptual], './mapa_conceptual.html')

# Tarea

La tarea consiste en diseñar e implementar una estrategia propia para la creación de mapas conceptuales utilizando el modelo de lenguaje Gemini. Todo el proceso debe integrarse en una función llamada crear_mapa_conceptual, la cual recibe como entrada un texto y devuelve como salida un diccionario de Python que codifica el mapa conceptual correspondiente.

A partir del archivo `documentos.json`, deberán generar un único mapa conceptual por cada texto, y almacenar cada resultado en un archivo JSON separado, siguiendo el formato ilustrado en los ejemplos anteriores.

Se recomienda experimentar con distintas estrategias de prompting y diferentes algoritmos. Pueden usar como referencia los prompts y piezas de código presentados en la sección previa. Además, el siguiente recurso puede serles útil como guía para diseñar prompts:
https://www.promptingguide.ai/

**Consejo**:
* Incorporen un paso adicional en la estrategia anterior que le pida al LLM mejorar un mapa conceptual previamente generado.

---

### Entrega

Cuando finalicen, suban su notebook, los prompts utilizados y todos los archivos con los mapas conceptuales generados a un repositorio público de GitHub. Envíen el enlace del repositorio al correo mat.torreta@gmail.com con el asunto:

* *InforSanLuis25-LLMs Entrega*

En el cuerpo del correo deben incluir:

* Nombre completo

* DNI

La entrega deberá realizarse antes del viernes 05 de diciembre a las 23:59 para aprobar el curso.

---

### Método a implementar

Implementen la mayor parte de su estrategia de creación de mapas conceptuales en el cuerpo de esta función. Pueden crear sus propias funciones auxiliares e invocarlas desde aquí si lo necesitan.

In [77]:
def crear_mapa_conceptual(texto: str) -> dict:
    """
    IMPORTANTE!

    Aquí deben implementar la estrategia para crear mapas conceptuales
    a partir de texto. Deberán coordinar las diferentes invocaciones
    al modelo de lenguaje y procesar correctamente las respuestas.
    """

    raise RuntimeError("Función no implementada.")